In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import math
import CRPS.CRPS as pscore
import datetime

pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.drbart_evaluation import *

In [2]:
log_name = 'train'
model_name = 'pcr'
n_processes = 20

with open('../transformed_event_logs/PCR_start_end_'+log_name+'.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

#test_event_log['time:timestamp'] = test_event_log['time:timestamp_complete']
test_event_log['case:concept:name'] = test_event_log['case:concept:name'].astype(str)


known_activities = ['Callback timeout', 'Export result', 'Export to EMS', 'Match patient data', 'Receive sample state', 'Send notification', 'Wait for plate validation', 'timeout']

likelihoods_A = None
likelihoods_A_S = None
likelihoods_A_S_AC = None

/tmp/ipykernel_31772/3978959522.py:6: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
N = 1000
import conduct_evaluation
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

In [4]:
drbart_model_A = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name/',
                     strict_parser=True)
evaluator_A = conduct_evaluation.ConductEvaluation(drbart_model_A, SampleOutcomes_DRBART_PCR_A,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A = evaluator_A.sample_cases(False, True)

  0%|                                                 | 0/4927 [00:00<?, ?it/s]

  0%|                                      | 1/4927 [00:29<40:38:54, 29.71s/it]

  0%|                                       | 4/4927 [00:29<7:46:56,  5.69s/it]

  0%|                                       | 6/4927 [00:31<4:55:59,  3.61s/it]

  0%|                                       | 7/4927 [00:32<4:01:11,  2.94s/it]

...

 99%|████████████████████████████████████▍| 4857/4927 [00:25<00:00, 199.63it/s]

 99%|████████████████████████████████████▌| 4877/4927 [00:25<00:00, 198.25it/s]

 99%|████████████████████████████████████▊| 4897/4927 [00:25<00:00, 198.49it/s]

100%|████████████████████████████████████▉| 4919/4927 [00:25<00:00, 201.26it/s]

100%|█████████████████████████████████████| 4927/4927 [00:25<00:00, 189.57it/s]

In [5]:
np.mean([v.ln() for v in likelihoods_A[0].values()])

Decimal('0.6620809211375778834740296949')

In [6]:
np.mean(get_pscores(likelihoods_A))

np.float64(12469.142926741044)

In [7]:
drbart_model_A_S = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day/',
                     strict_parser=True)
evaluator_A_S = conduct_evaluation.ConductEvaluation(drbart_model_A_S, SampleOutcomes_DRBART_PCR_A_S,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S = evaluator_A_S.sample_cases(False, True)

  0%|                                                 | 0/4927 [00:00<?, ?it/s]

  0%|                                      | 1/4927 [00:28<39:23:49, 28.79s/it]

  0%|                                      | 2/4927 [00:31<18:16:38, 13.36s/it]

  0%|                                       | 4/4927 [00:32<7:13:10,  5.28s/it]

  0%|                                       | 6/4927 [00:33<4:06:16,  3.00s/it]

...

 99%|████████████████████████████████████▍| 4859/4927 [00:30<00:00, 163.62it/s]

 99%|████████████████████████████████████▌| 4876/4927 [00:30<00:00, 143.32it/s]

 99%|████████████████████████████████████▊| 4900/4927 [00:30<00:00, 167.52it/s]

100%|████████████████████████████████████▉| 4918/4927 [00:30<00:00, 168.70it/s]

100%|█████████████████████████████████████| 4927/4927 [00:30<00:00, 159.89it/s]

In [8]:
np.mean([v.ln() for v in likelihoods_A_S[0].values()])

Decimal('0.7977026150566703455027593120')

In [9]:
np.mean(get_pscores(likelihoods_A_S))

np.float64(11482.606107497237)

In [10]:
drbart_model_A_S_D = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_day-of-week/',
                     strict_parser=True)
evaluator_A_S_D = conduct_evaluation.ConductEvaluation(drbart_model_A_S_D, SampleOutcomes_DRBART_PCR_A_S_D,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'day_of_week' : 'day_of_week',
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_D = evaluator_A_S_D.sample_cases(False, True)

  0%|                                                 | 0/4927 [00:00<?, ?it/s]

  0%|                                      | 1/4927 [00:45<61:40:34, 45.07s/it]

  0%|                                      | 2/4927 [00:47<27:35:08, 20.16s/it]

  0%|                                       | 5/4927 [00:48<8:11:04,  5.99s/it]

  0%|                                       | 7/4927 [00:51<5:31:22,  4.04s/it]

...

 98%|████████████████████████████████████▍| 4849/4927 [00:25<00:00, 191.86it/s]

 99%|████████████████████████████████████▌| 4869/4927 [00:26<00:00, 191.49it/s]

 99%|████████████████████████████████████▋| 4889/4927 [00:26<00:00, 193.45it/s]

100%|████████████████████████████████████▊| 4909/4927 [00:26<00:00, 193.52it/s]

100%|█████████████████████████████████████| 4927/4927 [00:26<00:00, 186.93it/s]

In [11]:
np.mean([v.ln() for v in likelihoods_A_S_D[0].values()])

Decimal('-3.963399753774232975712613254')

In [12]:
np.mean(get_pscores(likelihoods_A_S_D))

np.float64(9672.698978509874)

In [13]:
drbart_model_A_S_AC = DRBART(parser_dir = '../../../models/'+model_name+'/concept-name_seconds-in-day_activity-count/',
                     strict_parser=True)
evaluator_A_S_AC = conduct_evaluation.ConductEvaluation(drbart_model_A_S_AC, SampleOutcomes_DRBART_PCR_A_S_AC,
                                                   {
                                                        'activity_key' : 'concept:name',
                                                        'known_activities' : known_activities,
                                                   },
                                     test_event_log, n=N, n_processes=n_processes)
likelihoods_A_S_AC = evaluator_A_S_AC.sample_cases(False, True)

  0%|                                                 | 0/4927 [00:00<?, ?it/s]

  0%|                                      | 1/4927 [00:33<45:58:22, 33.60s/it]

  0%|                                      | 2/4927 [00:36<21:19:46, 15.59s/it]

  0%|                                       | 5/4927 [00:39<6:52:25,  5.03s/it]

  0%|                                       | 6/4927 [00:40<5:42:14,  4.17s/it]

...

 98%|████████████████████████████████████▍| 4844/4927 [00:25<00:00, 219.26it/s]

 99%|████████████████████████████████████▌| 4867/4927 [00:25<00:00, 206.40it/s]

 99%|████████████████████████████████████▋| 4889/4927 [00:25<00:00, 206.97it/s]

100%|████████████████████████████████████▉| 4911/4927 [00:25<00:00, 204.16it/s]

100%|█████████████████████████████████████| 4927/4927 [00:26<00:00, 189.30it/s]

In [14]:
cases = test_event_log['case:concept:name'].unique()
case_data = dict([(case_name, evaluator_A_S_AC.get_case_data(case_name)) for case_name in cases])

In [15]:
np.mean([v.ln() for v in likelihoods_A_S_AC[0].values()])

Decimal('0.2203403479601634194793056365')

In [16]:
np.mean(get_pscores(likelihoods_A_S_AC))

np.float64(15703.050887845317)

In [17]:
results = {
    'drbart_model_A' : likelihoods_A,
    'drbart_model_A_S' : likelihoods_A_S,
    'drbart_model_A_S_D' : likelihoods_A_S_D,
    'drbart_model_A_S_AC' : likelihoods_A_S_AC
}
with open('./pcr_dr_bart_evaluation_'+log_name+'.pickle', 'wb') as handle:
    pickle.dump(results, handle)